# Evaluation of trained models

In [1]:
from prettytable import PrettyTable
from typing import Optional, Dict, Union, Literal
import torch
import pytorch_lightning as pl
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

from HECKTOR_Dataset import Mode, Modality
from HECKTOR_DataModule import HECKTOR_DataModule
from HECKTOR_Model import HECKTOR_Model

In [3]:
# Testing hyperparameters:
DATA_PATH = "/home/dzban112/HECKTOR/Data/"
CHECKPOINTS_PATH="/home/dzban112/HECKTOR/ckpt/"
BATCH_SIZE = 16
NUM_WORKERS = 4
SELECTED_TRAIN_TRANSFORMS = ["elastic", "histogram", "crop"] # Test images will be also cropped to specified ROI. 

In [4]:
def model_evaluation(model_name:str, version:Optional[str]=None):
    """
    Evaluates Hecktor model on the test set.
    """
    res={}
    def _setup(fold:Union[int, Literal["all"]]=1):
        # DataModule setup:
        dm = HECKTOR_DataModule(
            fold=fold,
            data_path=DATA_PATH,
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            selected_train_transforms=SELECTED_TRAIN_TRANSFORMS,
            modality=MODALITY
        )
        
        if version is None:
            ckpt_path = CHECKPOINTS_PATH+model_name+f"_{fold}"+".ckpt"
        else:
            ckpt_path = CHECKPOINTS_PATH+model_name+f"_{fold}"+f"-{version}"+".ckpt"
        # Model setup:
        model = HECKTOR_Model.load_from_checkpoint(ckpt_path)
        # Trainer setup:
        torch.set_float32_matmul_precision('medium')
        trainer = pl.Trainer(accelerator="gpu", devices=1, precision=32, logger=False)
        
        return dm, model, trainer

    # Evaluating models trained on 5CV training sets:
    for fold in range(1, 6):
        dm, model, trainer = _setup(fold=fold)
        model.eval()
        res[f"fold_{fold}"] = trainer.test(model, dm)
    # Evaluating model trained on the whole training set:
    fold="all"
    dm, model, trainer = _setup(fold=fold)
    model.eval()
    res[f"fold_{fold}"] = trainer.test(model, dm)
        
    return res


def compute_results(raw_results:Dict[str, list]):
    c_index = []
    test_loss = []
    folds = []

    for fold in raw_results.keys():
        curr_c_index = raw_results[fold][0]["test_C-index"]
        curr_test_loss = raw_results[fold][0]["test_loss"]
        c_index.append(curr_c_index)
        test_loss.append(curr_test_loss)
        folds.append(f"Fold {fold.split('_')[1]}")

    df = pd.DataFrame({"Fold": folds, "C-index": c_index, "Test loss": test_loss})
    
    table = PrettyTable()
    table.field_names = df.columns.tolist()

    for row in df.itertuples(index=False):
        table.add_row(row)

    for field in df.columns:
        table.align[field] = 'l'
    
    return table

### DINO_ViTs8 - only CT

In [9]:
MODALITY = Modality.CT
res = model_evaluation("dino_vits8_21")

/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  6.71it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index            0.5510213971138
        test_loss           2.4985618591308594
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  6.83it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5292801260948181
        test_loss            2.873896598815918
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  6.92it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5512645840644836
        test_loss            2.474199056625366
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  6.53it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5261673331260681
        test_loss           2.4896223545074463
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  6.25it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.4923638105392456
        test_loss           2.5493812561035156
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [10]:
compute_results(res)

Fold,C-index,Test loss
Fold 1,0.5510213971138,2.4985618591308594
Fold 2,0.5292801260948181,2.873896598815918
Fold 3,0.5482490062713623,2.4760258197784424
Fold 4,0.5512645840644836,2.474199056625366
Fold 5,0.5261673331260681,2.4896223545074463
Fold all,0.4923638105392456,2.5493812561035156


### DINO_ViTs8 - Both CT and PET

In [11]:
MODALITY = Modality.BOTH
res = model_evaluation("dino_vits8_22")

/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:10<00:00,  2.03it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index           0.550340473651886
        test_loss           3.0010292530059814
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.37it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.45977625250816345
        test_loss           3.5156638622283936
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.47it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5762159824371338
        test_loss            2.418848752975464
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.38it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5441634058952332
        test_loss           2.5420284271240234
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.38it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5718871355056763
        test_loss            2.449272394180298
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [12]:
compute_results(res)

Fold,C-index,Test loss
Fold 1,0.550340473651886,3.0010292530059814
Fold 2,0.45977625250816345,3.5156638622283936
Fold 3,0.4816147983074188,3.0514771938323975
Fold 4,0.5762159824371338,2.418848752975464
Fold 5,0.5441634058952332,2.5420284271240234
Fold all,0.5718871355056763,2.449272394180298


### DINO_ViTb8 - only CT

In [14]:
MODALITY = Modality.CT
res = model_evaluation("dino_vitb8_23")

/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.37it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5022373795509338
        test_loss           2.4859931468963623
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:05<00:00,  3.79it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index           0.477237343788147
        test_loss            2.483219623565674
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:05<00:00,  3.75it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5856517553329468
        test_loss            2.438627243041992
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:05<00:00,  3.77it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5712548494338989
        test_loss           2.4892852306365967
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:05<00:00,  3.77it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5383025407791138
        test_loss           2.4777936935424805
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [15]:
compute_results(res)

Fold,C-index,Test loss
Fold 1,0.5022373795509338,2.4859931468963623
Fold 2,0.477237343788147,2.483219623565674
Fold 3,0.5695525407791138,2.4494426250457764
Fold 4,0.5856517553329468,2.438627243041992
Fold 5,0.5712548494338989,2.4892852306365967
Fold all,0.5383025407791138,2.4777936935424805


### DINO_ViTb8 - Both CT and PET

In [17]:
MODALITY = Modality.BOTH
res = model_evaluation("dino_vitb8_24")

/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.36it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5789883136749268
        test_loss           2.4143447875976562
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.45it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5616731643676758
        test_loss            2.450225353240967
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.50it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5625486373901367
        test_loss           2.4335992336273193
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.42it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5580739378929138
        test_loss            2.49345326423645
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████| 22/22 [00:06<00:00,  3.38it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_C-index          0.5697470903396606
        test_loss            2.467991352081299
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [18]:
compute_results(res)

Fold,C-index,Test loss
Fold 1,0.5789883136749268,2.4143447875976562
Fold 2,0.5616731643676758,2.450225353240967
Fold 3,0.5222276449203491,2.4687044620513916
Fold 4,0.5625486373901367,2.4335992336273193
Fold 5,0.5580739378929138,2.49345326423645
Fold all,0.5697470903396606,2.467991352081299


# CACHE:

In [38]:
def model_evaluation(model_name:str, version:Optional[str]=None):
    res={}
    for fold in range(1,6):
        print(f"fold: {fold}")
        dm = HECKTOR_DataModule(
            fold=fold,
            data_path=DATA_PATH,
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            selected_train_transforms=SELECTED_TRAIN_TRANSFORMS,
            modality=MODALITY
        )

        if version is None:
            ckpt_path = CHECKPOINTS_PATH+model_name+f"_{fold}"+".ckpt"
        else:
            ckpt_path = CHECKPOINTS_PATH+model_name+f"_{fold}"+f"-{version}"+".ckpt"
        model = HECKTOR_Model.load_from_checkpoint(ckpt_path)
        torch.set_float32_matmul_precision('medium')
        trainer = pl.Trainer(accelerator="gpu", devices=1, precision=32, logger=False)
        model.eval()
        res[f"fold_{fold}"] = trainer.test(model, dm)
    return res


def compute_results(raw_results:Dict[str, list]):
    c_index = []
    test_loss = []
    folds = []

    for fold in raw_results.keys():
        curr_c_index = raw_results[fold][0]["C-index"]
        curr_test_loss = raw_results[fold][0]["test_loss"]
        c_index.append(curr_c_index)
        test_loss.append(curr_test_loss)
        folds.append(f"Fold {fold}")

    df = pd.DataFrame({"Fold": folds, "C-index": c_index, "Test loss": test_loss})
    df.loc[len(df)] = ['Mean Accuracy', mean_ACC]
    
    table = PrettyTable()
    table.field_names = df.columns.tolist()

    for row in df.itertuples(index=False):
        table.add_row(row)

    for field in df.columns:
        table.align[field] = 'l'
    
    return table

In [39]:
res = model_evaluation("dino_vits16_1")

fold: 1


/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /lu/tetyda/home/dzban112/my_env/lib/python3.10/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                                                        | 0/? [00:00<?, ?it/s]

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/lu/tetyda/home/dzban112/my_env/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/lu/tetyda/home/dzban112/HECKTOR/HECKTOR_Dataset.py", line 126, in __getitem__
    return {"pid": pid, "crop": crops[self.modality.value], "labels": labels.to_numpy().astype(float)}
KeyError: 'CT'


In [22]:
res

{'fold_1': [{'test_loss': 2.5505716800689697, 'C-index': 0.5551331043243408}],
 'fold_2': [{'test_loss': 2.398300886154175, 'C-index': 0.60495525598526}],
 'fold_3': [{'test_loss': 2.3287007808685303, 'C-index': 0.4318334460258484}],
 'fold_4': [{'test_loss': 2.539806365966797, 'C-index': 0.506926953792572}],
 'fold_5': [{'test_loss': 2.395446300506592, 'C-index': 0.6455696225166321}]}